<a href="https://colab.research.google.com/github/kutouxiyiji/other/blob/master/Quora_alpha_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# continue with Quora_alpha_1, import libriary and mount google drive
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
from xgboost import XGBClassifier as xgbc
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))

nltk.download('punkt')
from tqdm import tqdm_notebook
from nltk.tokenize import word_tokenize

from tqdm import tqdm_notebook

from scipy.spatial.distance import *

from scipy.stats import kurtosis
from scipy.stats import skew

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
df_train = pd.read_csv('/content/drive/My Drive/Python/dataSet/Quora/train_df.csv')

In [0]:
train_qs1 = pd.Series(df_train['question1'].astype('str').values)
train_qs2 = pd.Series(df_train['question2'].astype('str').values)


In [0]:
train_qs1[:3], train_qs2[:3]

In [0]:
cloud = WordCloud(width=1440, height=1080).generate(" ".join(train_qs2.astype(str)))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.axis('off')

In [0]:
cloud = WordCloud(width=1440, height=1080).generate(" ".join(train_qs1.astype(str)))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.axis('off')

In [0]:
tvec = TfidfVectorizer(max_features = 1024,analyzer='word', stop_words='english')
tvec.fit(train_qs1)
X1 = tvec.transform(train_qs1)
X2 = tvec.transform(train_qs2)

In [0]:
X1.shape, X2.shape

In [0]:
y = df_train['is_duplicate'].values

In [0]:
np.mean(X1[0]),np.mean(X2[0]), np.mean(X1-X2)

In [0]:
(X1-X2).shape, y.shape

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X1-X2, y, test_size=0.33, random_state=42)

In [0]:
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['max_depth'] = 4

d_train = xgb.DMatrix(X_train, label=y_train)
bst = xgb.train(params, d_train)

In [0]:
xgb_model = xgbc()
xgb_model.fit(X= X_train,y = y_train)

In [0]:
y_pred = xgb_model.predict(X_test)

In [0]:
classification_report(y_test,y_pred)

In [0]:
cm=confusion_matrix(y_test,y_pred)
df_cm = pd.DataFrame(cm, range(2),range(2))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16})

there is a serious problem. The recall for duplicated class is very low. We try undersample the non-duplicated class first.

In [0]:
pos_train = X_train[y_train == 1]
neg_train = X_train[y_train == 0]
pos_train.shape, neg_train.shape

In [0]:
df_pos_train = df_train[df_train['is_duplicate']==1]
df_neg_train = df_train[df_train['is_duplicate']==0]
len(df_pos_train),len(df_neg_train)

In [0]:
df_neg_train_under = df_neg_train.sample(len(df_pos_train))
len(df_neg_train_under)

In [0]:
df_train_under = pd.concat([df_neg_train_under,df_pos_train], axis=0)
len(df_train_under)

In [0]:
train_qs1_under = pd.Series(df_train_under['question1'].astype('str').values)
train_qs2_under = pd.Series(df_train_under['question2'].astype('str').values)

In [0]:
tvec = TfidfVectorizer(max_features = 1024,analyzer='word', stop_words='english')
tvec.fit(train_qs1_under)
X1 = tvec.transform(train_qs1_under)
X2 = tvec.transform(train_qs2_under)

In [0]:
y = df_train_under['is_duplicate'].values

In [0]:
(X1-X2).shape, y.shape

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X1-X2, y, test_size=0.33, random_state=42)

In [0]:
xgb_model = xgbc()
xgb_model.fit(X= X_train,y = y_train)

In [0]:
y_pred = xgb_model.predict(X_test)
classification_report(y_test,y_pred)

In [0]:
cm=confusion_matrix(y_test,y_pred)
df_cm = pd.DataFrame(cm, range(2),range(2))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16})



---



---

try differnt vecrization

In [0]:
tvec2 = TfidfVectorizer(max_features = 128,analyzer='word', stop_words='english',max_df = 0.95, min_df=0.005)
tvec2.fit(train_qs1_under)
X1_2 = tvec2.transform(train_qs1_under)
X2_2 = tvec2.transform(train_qs2_under)
X_train, X_test, y_train, y_test = train_test_split(X1_2-X2_2, y, test_size=0.33, random_state=42)

In [0]:
xgb_model = xgbc()
xgb_model.fit(X= X_train,y = y_train)
y_pred = xgb_model.predict(X_test)
classification_report(y_test,y_pred)

In [0]:
df = pd.DataFrame()
df_X1 = pd.DataFrame(X1_2.toarray(), columns=tvec2.get_feature_names())
df_X2 = pd.DataFrame(X2_2.toarray())
df = pd.concat([df_X1, df_X2], axis=1)
# need re-run this cell if you already run the xgb model
df['question1'] = df_train_under['question1']
df['question2'] = df_train_under['question2']

we use the df as X

In [0]:
#(df_X1-df_X2).head()
#df2 = pd.DataFrame()
#df2=df_X1-df_X2
#df2.head()

In [0]:

len(df_X1), len(df_X2), len(df), df_X1.head(), df.head()

In [0]:
# add features
df['len_q1'] = df.question1.apply(lambda x: len(str(x)))
df['len_q2'] = df.question2.apply(lambda x: len(str(x)))
df['diff_len'] = df.len_q1 - df.len_q2
df['len_char_q1'] = df.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
df['len_char_q2'] = df.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
df['len_word_q1'] = df.question1.apply(lambda x: len(str(x).split()))
df['len_word_q2'] = df.question2.apply(lambda x: len(str(x).split()))
df['common_words'] = df.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)

In [0]:
df.drop(['question1', 'question2'], axis=1, inplace=True)
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
xgb_model = xgbc()
xgb_model.fit(X= X_train,y = y_train)
y_pred = xgb_model.predict(X_test)
classification_report(y_test,y_pred)



---

Try word2vec


---



In [0]:
def wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    stop_words = stopwords.words('english')
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    return model.wmdistance(q1, q2)
    
def norm_wmd(q1, q2):
    q1 = str(q1).lower().split()
    q2 = str(q2).lower().split()
    stop_words = stopwords.words('english')
    q1 = [w for w in q1 if w not in stop_words]
    q2 = [w for w in q2 if w not in stop_words]
    return norm_model.wmdistance(q1, q2)
    
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    stop_words = stopwords.words('english')
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

In [0]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Python/dataSet/Quora/GoogleNews-vectors-negative300.bin.gz', binary=True)
#df['wmd'] = df.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

In [0]:
model.init_sims(replace=True)
df['wmd'] = df.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)

In [0]:
norm_model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/My Drive/Python/dataSet/Quora/GoogleNews-vectors-negative300.bin.gz', binary=True)
norm_model.init_sims(replace=True)

In [0]:
question1_vectors = np.zeros((df.shape[0], 300))
for i, q in enumerate(tqdm_notebook(df.question1.values)):
    question1_vectors[i, :] = sent2vec(q)
    
question2_vectors  = np.zeros((df.shape[0], 300))
for i, q in enumerate(tqdm_notebook(df.question2.values)):
    question2_vectors[i, :] = sent2vec(q)
    


In [0]:

df['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors), np.nan_to_num(question2_vectors))]
df['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
df['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
df['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
df['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

In [0]:
df.drop(['question1', 'question2'], axis=1, inplace=True)
#df = df[pd.notnull(df['cosine_distance'])]
#df = df[pd.notnull(df['jaccard_distance'])]
X = df



In [0]:
y = df_train_under['is_duplicate'].values
X.shape, y.shape

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
import xgboost as xgb

model_xgb2 = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)

In [0]:
model_xgb2.fit(X= X_train,y = y_train)


In [0]:
filename = '/content/drive/My Drive/Python/dataSet/Quora/word2vec_1.model'
pickle.dump(model_xgb2, open(filename, 'wb'))

In [0]:
prediction = model_xgb2.predict(X_test)
cm = confusion_matrix(y_test, prediction)  
print(cm)  


In [0]:
df_cm = pd.DataFrame(cm, range(2),range(2))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16})

In [0]:
classification_report(y_test,prediction)

In [0]:
print('Accuracy', accuracy_score(y_test, prediction))
print(classification_report(y_test, prediction))

In [0]:

'''df.drop(['question1', 'question2'], axis=1, inplace=True)
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
xgb_model = xgbc()
xgb_model.fit(X= X_train,y = y_train)
y_pred = xgb_model.predict(X_test)
classification_report(y_test,y_pred)'''
